# Импорт

In [ ]:
import joblib

import pandas as pd
import numpy as np

import keras_nlp
from tensorflow import keras
from keras import losses, optimizers

from urllib.parse import unquote

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

!git clone https://github.com/mitramir55/Kaggle_NLP_competition.git
train = pd.read_csv('Kaggle_NLP_competition/train.csv')
test = pd.read_csv('Kaggle_NLP_competition/test.csv')


# 1. Загрузка TF-IDF векторизатор
vectorizer = joblib.load("/content/drive/MyDrive/Colab Notebooks/tfidf_vectorizer.joblib")
# 2. Загрузка модели логистической регрессии
logreg_model = joblib.load("/content/drive/MyDrive/Colab Notebooks/logreg_model.joblib")

# 3. Загрузка модели DistilBERT
distilbert_model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/saved_model_disaster_tweets.keras")

Cloning into 'Kaggle_NLP_competition'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 80 (from 1)
Receiving objects: 100% (81/81), 11.05 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (36/36), done.


/usr/local/lib/python3.11/dist-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 210 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
!git clone https://github.com/mitramir55/Kaggle_NLP_competition.git
test_target = pd.read_csv('Kaggle_NLP_competition/perfect_submission.csv')
test_target = test_target.drop(['id'],axis=1)
test['target'] = test_target['target']
test.shape

# Обработка данных для LogReg

In [ ]:
def drop_column_for_lr(df):
  df = df.copy()

  df = df.drop_duplicates(subset='text')
  df = df.drop(['id', 'location'], axis=1)
  df['keyword'] = df['keyword'].fillna('unknown')
  return df

# df_train_for_lr = drop_column_for_lr(train)
# df_test_for_lr = drop_column_for_lr(test)

In [ ]:
import re
import spacy

# Загрузить модель spaCy English (с отключенным парсером и NER для более высокой производительности)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def clean_text_spacy(text):
    text = text.lower() # Преобразование в нижний регистр
    text = re.sub(r"http\S+|www\S+|@\w+", "", text) # Удаление URL-адресов и упоминаний
    text = re.sub(r"#(\w+)", r"\1", text) # Упрошение хэштегов, удалив символ «#».
    text = re.sub(r"[^a-zA-Z\s]", "", text) # Удаление специальных символов и цифр
    text = re.sub(r"\s+", " ", text).strip() # Удаление лишних пробелов

    # Лемматизировать и удалить стоп-слова, сохранить токены длиннее 2 символов
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and len(token) > 2]

    return " ".join(tokens)

# df_train_for_lr['cleaned_text'] = df_train_for_lr['text'].apply(clean_text_spacy)
# df_test_for_lr['cleaned_text'] = df_test_for_lr['text'].apply(clean_text_spacy)

# df_train_for_lr.head()

In [ ]:
from urllib.parse import unquote

def clear_df_unquote(df):
  df['keyword'] = df['keyword'].apply(lambda x: unquote(x.replace('+', ' ')))
  df['final_text'] = df['keyword'] + ' ' + df['cleaned_text']
  df = df.drop(['keyword', 'text', 'cleaned_text'], axis=1)
  df = df[['final_text', 'target']]

  return df

# df_train_for_lr = clear_df_unquote(df_train_for_lr)
# df_test_for_lr = clear_df_unquote(df_test_for_lr)

In [ ]:
def df_vectorize_and_split(df, test_size=0.2, random_state=42):
  X_for_lr = vectorizer.transform(df['final_text'])
  y_for_lr = df['target']
  # Разделить данные на обучающие и проверочные наборы
  X_train, X_val, y_train, y_val = train_test_split(
      X_for_lr, y_for_lr,
      test_size=test_size,
      stratify=y_for_lr,           # Стратифицированное разделение для сохранения распределения классов
      random_state=random_state
  )
  return X_train, X_val, y_train, y_val

# X_train, X_val, y_train, y_val = df_vectorize_and_split(df_train_for_lr)

# Обработка данных для DistilBert

In [ ]:
def drop_column_for_db(train):
  df = train.copy()

  df = df.drop_duplicates(subset='text')
  df = df.drop(['id', 'location'], axis=1)
  df['keyword'] = df['keyword'].fillna('unknown')

  df['keyword'] = df['keyword'].apply(lambda x: unquote(x.replace('+', ' ')))

  df['final_text'] = df['keyword'] + ' ' + df['text']

  df = df.drop(['keyword', 'text'], axis=1)

  return df

# df_train_for_bert = drop_сolumn_for_db(train)
# df_test_for_bert = drop_сolumn_for_db(test)

In [ ]:
def df_split(df, test_size=0.2, random_state=42):
    X = df['final_text']
    y = df['target']

    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        stratify=y,
        test_size=test_size,
        random_state=random_state)

    return X_train, X_val, y_train, y_val

# X_train, X_val, y_train, y_val = df_split(df_train_for_bert)

In [ ]:
y_train_pred_prob_by_distilbert = distilbert_model.predict(X_train)

In [ ]:
y_val_pred_probs_by_distilber = distilbert_model.predict(X_val)

In [ ]:
y_train_pred_by_distilbert = np.argmax(y_train_pred_prob_by_distilbert, axis=1)
y_val_pred_by_distilbert = np.argmax(y_val_pred_probs_by_distilber, axis=1)

# Результаты LogRec

In [ ]:
y_pred_train_by_lr = logreg_model.predict(X_train)
y_pred_val_by_lr = logreg_model.predict(X_val)

print("Отчет о классификации тренировочного набора:")
print(classification_report(y_train, y_pred_train_by_lr))
print(' ')
print("Отчет о классификации валидационного набора:")
print(classification_report(y_val, y_pred_val_by_lr))

# Результаты DisrilBert

In [ ]:
print("Отчет о классификации тренировочного набора:")
print(classification_report(y_train, y_train_pred_by_distilbert))
print(' ')
print("Отчет о классификации валидационного набора:")
print(classification_report(y_val, y_val_pred_by_distilbert))

# Ансамблирование

In [ ]:
train_for_lr = drop_column_for_lr(train)
train_for_lr['cleaned_text'] = train_for_lr['text'].apply(clean_text_spacy)
train_for_lr = clear_df_unquote(train_for_lr)
X_train_lr = vectorizer.transform(train_for_lr['final_text'])
y_train = train_for_lr['target']

train_for_db = drop_сolumn_for_db(train)
X_train_db = train_for_db['final_text']

test_for_lr = drop_column_for_lr(test)
test_for_lr['cleaned_text'] = test_for_lr['text'].apply(clean_text_spacy)
test_for_lr = clear_df_unquote(test_for_lr)
X_test_lr = vectorizer.transform(test_for_lr['final_text'])
y_test = test_for_lr['target']

test_for_db = drop_сolumn_for_db(test)
X_test_db = test_for_db['final_text']

In [ ]:
train_pred_prob_lr = logreg_model.predict_proba(X_train_lr)
val_pred_prob_lr = logreg_model.predict_proba(X_test_lr)

train_pred_prob_db = distilbert_model.predict(X_train_db)
val_pred_prob_db = distilbert_model.predict(X_test_db)

# min_val = np.min(y_train_pred_prob_by_distilbert)
# max_val = np.max(y_train_pred_prob_by_distilbert)
# y_train_pred_prob_by_distilbert_normalized = (y_train_pred_prob_by_distilbert - min_val) / (y_train_pred_prob_by_distilbert - min_val)



In [ ]:
w1, w2 = 0.2, 0.9
proba_wa = w1 * val_pred_prob_lr + w2 * val_pred_prob_db
preds_wa = proba_wa.argmax(axis=1)

print(classification_report(y_test, preds_wa))

In [ ]:
X_meta_train = np.hstack([
    train_pred_prob_lr,
    train_pred_prob_db
])
y_meta_train = y_train.values

X_meta_test = np.hstack([
    val_pred_prob_lr,
    val_pred_prob_db
])
y_meta_test = y_test.values

In [ ]:
# MLP
meta_mlp = MLPClassifier(
    hidden_layer_sizes=(32,16,8),
    max_iter=200,
    random_state=42)
meta_mlp.fit(X_meta_train, y_meta_train)
preds_meta = meta_mlp.predict(X_meta_test)


print("Отчет. Стэкинг, MLPClassifier, валидационный набор:")
print(classification_report(y_meta_test, preds_meta))

In [ ]:
from xgboost import XGBClassifier

# XGBoost
meta_xgb = XGBClassifier(
    n_estimators=300,
    max_depth=3,
    learning_rate=0.001,
    random_state=42,
    eval_metric='logloss'
)
meta_xgb.fit(X_meta_train, y_meta_train)

preds_xgb = meta_xgb.predict(X_meta_test)
print("Stacking XGBoost Report:")
print(classification_report(y_meta_test, preds_xgb))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# RandomForest
meta_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
    random_state=42
)
meta_rf.fit(X_meta_train, y_meta_train)

preds_rf = meta_rf.predict(X_meta_test)
print("Stacking RandomForest Report:")
print(classification_report(y_meta_test, preds_rf))

# Функции тестирования

In [ ]:
own_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/own_dataset.csv')

In [ ]:
def analyze_predictions(results_df, n_samples=2):
    print("\nОтчет о классификации:")
    print(classification_report(
        results_df['true_label'],
        results_df['predicted_label']))

    fp = results_df[(results_df['true_label'] == 0) & (results_df['predicted_label'] == 1)]
    fn = results_df[(results_df['true_label'] == 1) & (results_df['predicted_label'] == 0)]
    tp = results_df[(results_df['true_label'] == 1) & (results_df['predicted_label'] == 1)]
    tn = results_df[(results_df['true_label'] == 0) & (results_df['predicted_label'] == 0)]

    def print_samples(category, samples):
        print(f"\n{category} ({len(samples)} всего):")
        if len(samples) > 0:
            for i, row in samples.sample(min(n_samples, len(samples))).iterrows():
                print(f"Сообщение({i}):")
                print(f"\tТекст сообщения: {row['original_text']}")
                if row['true_label'] == 1:
                    print("\tИстинный: катастрофа")
                else:
                    print("\tИстинный: не катастрофа")

                if row['predicted_label'] == 1:
                    print("\tПредсказанный: катастрофа")
                else:
                    print("\tПредсказанный: не катастрофа")
        else:
            print("Нет примеров")

    print_samples("Ложноположительные", fp)
    print_samples("Ложноотрицательные", fn)
    print_samples("Истинноположительные", tp)
    print_samples("Истинноотрицательные", tn)

In [ ]:
def predict_text_by_lr(df):
    if isinstance(df, str):
        cleaned_tweet = clean_text_spacy(df)
        vectorized_tweet = vectorizer.transform([cleaned_tweet])
        logrec_pred = logreg_model.predict(vectorized_tweet)
        if logrec_pred == 0:
          print("Логическая регрессия предсказала что это: не катастрофа")
        elif logrec_pred == 1:
          print("Логическая регрессия предсказала что это: катастрофа")
        print("При вероятностях:", logreg_model.predict_proba(vectorized_tweet))

In [ ]:
def predict_text_by_db(df):
    if isinstance(df, str):
        distilbert_logit = distilbert_model.predict([df])
        distilbert_pred = distilbert_logit.argmax(axis=-1)
        if distilbert_pred == 0:
          print("DistilBERT предсказал что это: не катастрофа")
        elif distilbert_pred == 1:
          print("DistilBERT предсказал что это: катастрофа")
        print("С вероятностями: ", distilbert_logit)

In [ ]:
def predict_df_by_lr(df):
    df_for_lr = drop_column_for_lr(df)
    df_for_lr['cleaned_text'] = df_for_lr['text'].apply(clean_text_spacy)
    df_for_lr = clear_df_unquote(df_for_lr)

    X = vectorizer.transform(df_for_lr['final_text'])
    y = df_for_lr['target'].values

    pred_by_lr = logreg_model.predict(X)

    results_df = pd.DataFrame({
        'original_text': df['text'],
        'true_label': y,
        'predicted_label': pred_by_lr
    })

    return results_df

In [ ]:
def predict_df_by_db(df):
    df_for_bert = drop_column_for_db(df)

    X = df_for_bert['final_text']
    y = df_for_bert['target']

    pred_prob_by_distilbert = distilbert_model.predict(X)
    pred_by_distilbert = np.argmax(pred_prob_by_distilbert, axis=1)

    results_df = pd.DataFrame({
        'original_text': X,
        'true_label': y,
        'predicted_label': pred_by_distilbert
        })

    return results_df

In [ ]:
results_lr = predict_df_by_lr(own_df)
analyze_predictions(results_lr)


Отчет о классификации:
              precision    recall  f1-score   support

           0       0.65      0.79      0.71        14
           1       0.80      0.67      0.73        18

    accuracy                           0.72        32
   macro avg       0.72      0.73      0.72        32
weighted avg       0.73      0.72      0.72        32


Ложноположительные (3 всего):
Сообщение(27):
	Текст сообщения: Festive namaz at the Moscow Cathedral Mosque near Prospekt Mira on the occasion of Eid al-Adha
	Истинный: не катастрофа
	Предсказанный: катастрофа
Сообщение(30):
	Текст сообщения: A fat trained pig surprised Muscovites. A grunting mini-caban from Southern Butovo has delighted passers.
	Истинный: не катастрофа
	Предсказанный: катастрофа

Ложноотрицательные (6 всего):
Сообщение(8):
	Текст сообщения: The first summer thunderstorm came to Moscow - lightning struck the Ostankino TV Tower
	Истинный: катастрофа
	Предсказанный: не катастрофа
Сообщение(1):
	Текст сообщения: Smoke occurre

In [ ]:
results_db = predict_df_by_db(own_df)
analyze_predictions(results_db)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 936ms/step

Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      0.79      0.88        14
           1       0.86      1.00      0.92        18

    accuracy                           0.91        32
   macro avg       0.93      0.89      0.90        32
weighted avg       0.92      0.91      0.90        32


Ложноположительные (3 всего):
Сообщение(3):
	Текст сообщения: unknown The situation with coronavirus in Russia is stable and controlled, there is no information about the emergence of a new virus
	Истинный: не катастрофа
	Предсказанный: катастрофа
Сообщение(10):
	Текст сообщения: unknown The coming week in the Moscow region is expected to be much colder than the outgoing one, with average temperatures 1-2 degrees below normal
	Истинный: не катастрофа
	Предсказанный: катастрофа

Ложноотрицательные (0 всего):
Нет примеров

Истинноположительные (18 всего):
Сообщение(11):
	Текст сообщения: unknown Rocket d

In [ ]:
text = "An unidentified man with a hammer and stones attacked a synagogue in Sochi, filming everything on his phone"

predict_text_by_db(text)
print('')
predict_text_by_lr(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step
DistilBERT предсказал что это: катастрофа
С вероятностями:  [[-1.063919   1.0191315]]

Логическая регрессия предсказала что это: не катастрофа
При вероятностях: [[0.68480423 0.31519577]]
